In [1]:
import torch
import numpy as np
from utils import get_dataset, get_net, get_strategy
import pickle

In [2]:
samples = np.load("samples_bert.npy")

In [3]:
samples

array([  0, 100, 161, 221, 282, 344, 409, 474])

In [4]:
# fix random seed
# np.random.seed(1)
# torch.manual_seed(1)
torch.backends.cudnn.enabled = False
name = "BERT"
strat = "MarginSampling"
# device
acc_list_kc_list = []
agr_list_kc_list = []
kl_list_kc_list = []

for i in range(10):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # dataset = Data(unlab_x, unlab_y, test_x, test_y, cifar_handler) 
    dataset = get_dataset(name)    # load dataset
    net = get_net(name, device, y_num = 10)            # load network
    strategy = get_strategy(strat)(dataset, net)  # load strategy
    initial = samples[1]
    # start experiment
    dataset.initialize_labels(initial)
    print(f"number of labeled pool: {initial}")
    print(f"number of unlabeled pool: {dataset.n_pool-initial}")
    print(f"number of testing pool: {dataset.n_test}")
    print()

    # round 0 accuracy
    print("Round 0")
    strategy.train()
    preds, agr, kl = strategy.predict(dataset.get_test_data())
    acc_list_kc = []
    agr_list_kc = []
    kl_list_kc = []
    acc = dataset.cal_test_acc(preds)*100
    acc_list_kc.append(acc)
    agr_list_kc.append(agr)
    kl_list_kc.append(kl)
    print(f"Round 0 testing accuracy: {acc} %")
    print("Samples:", samples[1])
    for rd in range(1, len(samples)-1):
        print(f"Round {rd}")
        budget = samples[rd+1] - samples[rd]
        # query
        query_idxs = strategy.query(budget)
        # update labels
        strategy.update(query_idxs)
        strategy.train()

        # calculate accuracy
        preds, agr, kl = strategy.predict(dataset.get_test_data())
        acc = dataset.cal_test_acc(preds)*100
        print('\n')
        print(f"Round {rd} testing accuracy: {acc} %")
        acc_list_kc.append(acc)
        agr_list_kc.append(agr)
        kl_list_kc.append(kl)
        print("Samples:", samples[rd+1])
        torch.save(strategy.net.clf.state_dict(), "./extracted_models/"+name+"_"+strat+"_"+str(i)+str(3)+".pt")
    acc_list_kc_list.append(acc_list_kc)
    agr_list_kc_list.append(agr_list_kc)
    kl_list_kc_list.append(kl_list_kc)

    with open("./results/acc"+name+strat+str(3), "wb") as fp:
      pickle.dump(acc_list_kc_list, fp)
    with open("./results/agr"+name+strat+str(3), "wb") as fp:
      pickle.dump(agr_list_kc_list, fp)
    with open("./results/kl"+name+strat+str(3), "wb") as fp:
      pickle.dump(kl_list_kc_list, fp)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


number of labeled pool: 100
number of unlabeled pool: 7000
number of testing pool: 290

Round 0


  0%|                                                                         | 0/3 [00:00<?, ?it/s]

Train accuracy =  tensor(0.1833, device='cuda:0')
saving model


 33%|█████████████████████▋                                           | 1/3 [00:38<01:16, 38.42s/it]

...

Train accuracy =  tensor(1.2526, device='cuda:0')
saving model


 67%|███████████████████████████████████████████▎                     | 2/3 [01:48<00:53, 53.78s/it]

Train accuracy =  tensor(1.2842, device='cuda:0')
saving model


100%|█████████████████████████████████████████████████████████████████| 3/3 [02:10<00:00, 43.45s/it]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 37/37 [00:07<00:00,  4.98it/s]




Round 6 testing accuracy: 89.3103448275862 %
Samples: 474


In [7]:
acc_list_kc_list

[[18.96551724137931,
  31.379310344827587,
  40.3448275862069,
  43.79310344827586,
  63.793103448275865,
  66.89655172413794,
  86.89655172413792],
 [28.27586206896552,
  28.965517241379313,
  51.724137931034484,
  61.03448275862069,
  63.10344827586207,
  73.44827586206897,
  88.62068965517241],
 [21.03448275862069,
  29.655172413793103,
  23.103448275862068,
  40.3448275862069,
  43.44827586206896,
  52.41379310344828,
  72.06896551724138],
 [19.72035742143245,
  33.13549699263876,
  22.243819965826287,
  37.66238265814081,
  47.03274332749708,
  50.73461473192791,
  71.67906336516073],
 [22.104088575454465,
  21.52644470583729,
  33.398893426641756,
  38.61883047201417,
  54.57053167863416,
  72.96922074882482,
  76.17152623292337],
 [20.344827586206897,
  23.448275862068964,
  32.758620689655174,
  41.03448275862069,
  57.58620689655173,
  73.44827586206897,
  78.27586206896552],
 [24.482758620689655,
  42.41379310344828,
  44.48275862068966,
  43.79310344827586,
  43.448275862068

In [11]:
np.mean(acc, axis = 0)

array([24.1824446 , 29.67309072, 35.21944375, 44.66260407, 54.71205164,
       66.163487  , 78.64712793])